# In Depth A/B Testing - Lab

## Introduction

In this lab, you'll explore a survey from Kaggle regarding budding data scientists. With this, you'll form some initial hypotheses, and test them using the tools you've acquired to date. 

## Objectives

You will be able to:
* Conduct t-tests and an ANOVA on a real-world dataset and interpret the results

## Load the Dataset and Perform a Brief Exploration

The data is stored in a file called **multipleChoiceResponses_cleaned.csv**. Feel free to check out the original dataset referenced at the bottom of this lab, although this cleaned version will undoubtedly be easier to work with. Additionally, meta-data regarding the questions is stored in a file name **schema.csv**. Load in the data itself as a Pandas DataFrame, and take a moment to briefly get acquainted with it.

> Note: If you can't get the file to load properly, try changing the encoding format as in `encoding='latin1'`

In [73]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as pyplot
import scipy.stats as stats
from statsmodels.stats.power import TTestIndPower
import seaborn as sns
import flatiron_stats
%matplotlib inline

import statsmodels.api as sm
from statsmodels.formula.api import ols

In [17]:
#Your code here

df = pd.read_csv('multipleChoiceResponses_cleaned.csv')

In [35]:
filtered_df = df[['AdjustedCompensation', 'FormalEducation']].dropna().reset_index(drop=True)

filtered_df.head()

,AdjustedCompensation,FormalEducation
0,250000.0,Master's degree
1,64184.8,Bachelor's degree
2,20882.4,Bachelor's degree
3,1483.9,Bachelor's degree
4,36634.4,Master's degree


## Wages and Education

You've been asked to determine whether education is impactful to salary. Develop a hypothesis test to compare the salaries of those with Master's degrees to those with Bachelor's degrees. Are the two statistically different according to your results?

> Note: The relevant features are stored in the 'FormalEducation' and 'AdjustedCompensation' features.

You may import the functions stored in the `flatiron_stats.py` file to help perform your hypothesis tests. It contains the stats functions that you previously coded: `welch_t(a,b)`, `welch_df(a, b)`, and `p_value(a, b, two_sided=False)`. 

Note that `scipy.stats.ttest_ind(a, b, equal_var=False)` performs a two-sided Welch's t-test and that p-values derived from two-sided tests are two times the p-values derived from one-sided tests. See the [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html) for more information.    

In [53]:
#Your code here
# H-0: no difference in wages based on educational attainment
# H-a: difference in wages based on educational attainment

print("Welch T-test: " + str(flatiron_stats.welch_t(\
                            filtered_df[filtered_df['FormalEducation'] == 'Master\'s degree']['AdjustedCompensation'],\
                            filtered_df[filtered_df['FormalEducation'] == 'Bachelor\'s degree']['AdjustedCompensation'])))

print("Welch Degrees of Freedom: " + str(flatiron_stats.welch_df(\
                            filtered_df[filtered_df['FormalEducation'] == 'Master\'s degree']['AdjustedCompensation'],\
                            filtered_df[filtered_df['FormalEducation'] == 'Bachelor\'s degree']['AdjustedCompensation'])))

print("p-value welch T-test: " + str(flatiron_stats.p_value_welch_ttest(\
                            filtered_df[filtered_df['FormalEducation'] == 'Master\'s degree']['AdjustedCompensation'],\
                            filtered_df[filtered_df['FormalEducation'] == 'Bachelor\'s degree']['AdjustedCompensation'],\
                                                                       two_sided=False)))


Welch T-test: 0.43786693335411514
Welch Degrees of Freedom: 1350.0828973008781
p-value welch T-test: 0.33077639451272445


## Wages and Education II

Now perform a similar statistical test comparing the AdjustedCompensation of those with Bachelor's degrees and those with Doctorates. If you haven't already, be sure to explore the distribution of the AdjustedCompensation feature for any anomalies. 

In [66]:
#Your code here
# 28,297,400,000; 95666.08
# filtered_df.quantile(.99)

print("Welch T-test: " + str(flatiron_stats.welch_t(\
                            filtered_df[filtered_df['FormalEducation'] == 'Doctoral degree']['AdjustedCompensation'],\
                            filtered_df[filtered_df['FormalEducation'] == 'Bachelor\'s degree']['AdjustedCompensation'])))

print("Welch Degrees of Freedom: " + str(flatiron_stats.welch_df(\
                            filtered_df[filtered_df['FormalEducation'] == 'Doctoral degree']['AdjustedCompensation'],\
                            filtered_df[filtered_df['FormalEducation'] == 'Bachelor\'s degree']['AdjustedCompensation'])))

print("p-value welch T-test: " + str(flatiron_stats.p_value_welch_ttest(\
                            filtered_df[filtered_df['FormalEducation'] == 'Doctoral degree']['AdjustedCompensation'],\
                            filtered_df[filtered_df['FormalEducation'] == 'Bachelor\'s degree']['AdjustedCompensation'],\
                                                                       two_sided=False)))


Welch T-test: 1.0081234695549772
Welch Degrees of Freedom: 966.0001919995985
p-value welch T-test: 0.15682381994720251


## Wages and Education III

Remember the multiple comparisons problem; rather than continuing on like this, perform an ANOVA test between the various 'FormalEducation' categories and their relation to 'AdjustedCompensation'.

In [78]:
#Your code here

formula = 'AdjustedCompensation ~ FormalEducation'
lm = ols(formula, filtered_df).fit()
table = sm.stats.anova_lm(lm, typ=2)
print(table)

                       sum_sq      df         F    PR(>F)
FormalEducation  6.540294e+17     6.0  0.590714  0.738044
Residual         7.999414e+20  4335.0       NaN       NaN


## Additional Resources

Here's the original source where the data was taken from:  
    [Kaggle Machine Learning & Data Science Survey 2017](https://www.kaggle.com/kaggle/kaggle-survey-2017)

## Summary

In this lab, you practiced conducting actual hypothesis tests on actual data. From this, you saw how dependent results can be on the initial problem formulation, including preprocessing!